In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from speedy_utils.all import *

setup_logger("D")

DEBUG    | utils_print.py:298 (setup_logger) - Logging set to DEBUG


In [38]:
import pandas as pd
from joblib import Memory

memory = Memory("./cachedir", verbose=1)

@memory.cache
def cached_b(a, x):
    print("Computing...")
    return a + 1  # Uses the instance attribute

class A:
    def __init__(self):
        self.a = 1

    def b(self, x):
        print("Computing...")
        return self.a + 1



In [3]:
# setup_logger("i")

# results = None
# results = multi_thread(
#     slow_func,
#     range(200),
#     workers=10,
#     desc="Demo",
#     verbose=True
# )